前置設定

In [3]:

# ===== 基礎 =====
import os, re, numpy as np, pandas as pd
from scipy import sparse as sp
from bs4 import BeautifulSoup

# Sklearn
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction import FeatureHasher
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import roc_auc_score
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer

# NLTK
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk import pos_tag, word_tokenize, ne_chunk

# 其他
from joblib import Parallel, delayed
from math import ceil

# ===== 路徑與隨機種子 =====
DATA_DIR = './dataset'
TRAIN_PATH = f'{DATA_DIR}/train.csv'
TEST_PATH  = f'{DATA_DIR}/test.csv'
OUT_DIR = './output'
os.makedirs(OUT_DIR, exist_ok=True)
np.random.seed(42)

# ===== 速度開關（訓練時可先關慢模塊）=====
FAST_NO_NER = True        # 關 NER
FAST_NO_LDA = True        # 關 LDA 主題
FAST_NO_TEXTSTAT = True   # 關可讀性評分

# ===== NLTK 資源（缺什麼補什麼）=====
for rid, name in [
    ('tokenizers/punkt', 'punkt'),
    ('corpora/stopwords', 'stopwords'),
    ('corpora/wordnet', 'wordnet'),
    ('sentiment/vader_lexicon', 'vader_lexicon'),
    ('taggers/averaged_perceptron_tagger', 'averaged_perceptron_tagger'),
    ('chunkers/maxent_ne_chunker', 'maxent_ne_chunker'),
    ('corpora/words', 'words'),
]:
    try:
        nltk.data.find(rid)
    except LookupError:
        nltk.download(name)

# 全局 VADER（避免每條樣本重建）
try:
    VADER = SentimentIntensityAnalyzer()
except Exception:
    VADER = None


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\11958\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\11958\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


導入數據與切分

In [5]:
# 將 Popularity {-1,1} → {0,1}；並切出 train/val
df_all = pd.read_csv(TRAIN_PATH)
df_all['Popularity'] = (df_all['Popularity'].astype(int) == 1).astype(int)

TRAIN_SIZE = 26000
VAL_SIZE   = 1000

train_df = df_all.iloc[:TRAIN_SIZE].reset_index(drop=True)
val_df   = df_all.iloc[TRAIN_SIZE:TRAIN_SIZE+VAL_SIZE].reset_index(drop=True)

print(train_df.shape, val_df.shape)
train_df.head(2)


(26000, 3) (1000, 3)


,Id,Popularity,Page content
0,0,0,"<html><head><div class=""article-info""> <span c..."
1,1,1,"<html><head><div class=""article-info""><span cl..."


工具函數

In [8]:
# 小工具
_BODY_OPEN = re.compile(
    r'(?is)<\s*(section|div|article)\b[^>]*\b'
    r'(?:article-content|article-body|content-body|post-content)\b[^>]*>'
)
_MONTH = dict(jan='01', feb='02', mar='03', apr='04', may='05', jun='06',
              jul='07', aug='08', sep='09', oct='10', nov='11', dec='12')

def _norm(s: str) -> str:
    return re.sub(r'[\W]+', ' ', (s or '').lower()).strip()

def _slug(s: str) -> str:
    return re.sub(r'[^a-z0-9_]+', '', _norm(s).replace(' ', '_'))

def _bucket(n, edges):
    if n is None: return 'unk'
    for i in range(len(edges)-1):
        if edges[i] <= n < edges[i+1]:
            return f"b{edges[i]}_{edges[i+1]}"
    return f"b{edges[-1]}p"

def _aspect_bucket(w, h):
    if not w or not h: return 'unk'
    r = w / h
    if r < 0.9: return 'tall'
    if r < 1.2: return 'squareish'
    if r < 1.8: return 'landscape'
    return 'ultrawide'

def _img_size_bucket(w, h):
    if not w or not h: return 'unk'
    area = (w or 0) * (h or 0)
    if area < 80_000: return 'xs'
    if area < 230_000: return 'sm'
    if area < 920_000: return 'md'
    if area < 2_100_000: return 'lg'
    return 'xl'

def _parse_wh_from_src(src: str):
    if not src: return (None, None)
    m = re.search(r'/(\d{2,5})x(\d{2,5})/', src)
    return (int(m.group(1)), int(m.group(2))) if m else (None, None)

TRENDING_TOPICS = {
    'elon_musk', 'ai', 'climate_change', 'covid', 'blockchain', 'taiwan',
    'tesla', 'space', 'crypto', 'elections'
}

# LDA（可選）
def pretrain_lda(df, column='Page content', n_components=10, max_features=1000, max_text_len=500):
    def extract_text(html):
        if not isinstance(html, str) or not html.strip(): return ""
        m = _BODY_OPEN.search(html)
        header_html = html[:m.start()] if m else html
        soup = BeautifulSoup(header_html, 'html.parser')
        return ' '.join(soup.get_text().lower().split()[:max_text_len])

    corpus = [extract_text(x) for x in df[column].astype(str)]
    if not any(corpus): return None, None
    vec = CountVectorizer(max_features=max_features, stop_words='english')
    X = vec.fit_transform(corpus)
    lda = LatentDirichletAllocation(n_components=n_components, random_state=42)
    lda.fit(X)
    return vec, lda

lda_vectorizer, lda_model = (None, None)
if not FAST_NO_LDA:
    _df_lda = df_all.dropna(subset=['Page content']).astype({'Page content':'str'})
    lda_vectorizer, lda_model = pretrain_lda(_df_lda, 'Page content', n_components=10, max_features=1000)
    print("LDA pretrained")
else:
    print("LDA skipped by FAST_NO_LDA=True")


LDA skipped by FAST_NO_LDA=True


預處理函數

In [ ]:
# =========================
#  News Preprocessing + Feature Blocks (6‑tuple) — 对齐 Mashable 61 特征
# =========================

# ---- Imports ----
import re
import math
import numpy as np
import pandas as pd
import scipy.sparse as sp
from bs4 import BeautifulSoup
from joblib import Parallel, delayed
from urllib.parse import urlparse

# sklearn
from sklearn.feature_extraction.text import HashingVectorizer, FeatureHasher

# nltk（带安全降级）
try:
    import nltk
    from nltk.corpus import stopwords
    from nltk.stem.porter import PorterStemmer
    from nltk import pos_tag
    from nltk.tokenize import word_tokenize
    try:
        nltk.data.find('tokenizers/punkt')
    except LookupError:
        nltk.download('punkt', quiet=True)
    try:
        nltk.data.find('taggers/averaged_perceptron_tagger')
    except LookupError:
        nltk.download('averaged_perceptron_tagger', quiet=True)
    try:
        nltk.data.find('chunkers/maxent_ne_chunker')
        nltk.data.find('corpora/words')
        from nltk import ne_chunk
    except LookupError:
        nltk.download('maxent_ne_chunker', quiet=True)
        nltk.download('words', quiet=True)
        from nltk import ne_chunk
    try:
        from nltk.sentiment import SentimentIntensityAnalyzer
        nltk.data.find('sentiment/vader_lexicon')
    except LookupError:
        nltk.download('vader_lexicon', quiet=True)
    _NLTK_OK = True
except Exception:
    _NLTK_OK = False
    stopwords = None
    PorterStemmer = None
    pos_tag = None
    word_tokenize = None
    ne_chunk = None
    SentimentIntensityAnalyzer = None

# ---- 全局开关 ----
FAST_NO_NER = True      # True 可关闭 NER
FAST_NO_TEXTSTAT = True # True 可关闭 textstat 可读性

# ---- 可读性：textstat（可选）----
if not FAST_NO_TEXTSTAT:
    try:
        from textstat import flesch_reading_ease
        _HAS_TEXTSTAT = True
    except Exception:
        _HAS_TEXTSTAT = False
else:
    _HAS_TEXTSTAT = False

# ---- 停用词与词干 ----
if _NLTK_OK and stopwords is not None:
    try:
        STOP = set(stopwords.words('english'))
    except LookupError:
        nltk.download('stopwords', quiet=True)
        try:
            STOP = set(stopwords.words('english'))
        except Exception:
            STOP = set("""the a an and or not no to of in on for with by as at from be is are was were been being it this that these those he she they we you i if else then than when where who whom why what which into out up down over under again further here there all any both each few more most other some such nor only own same so too very can will just don don't should now""".split())
else:
    STOP = set("""the a an and or not no to of in on for with by as at from be is are was were been being it this that these those he she they we you i if else then than when where who whom why what which into out up down over under again further here there all any both each few more most other some such nor only own same so too very can will just don don't should now""".split())

porter = PorterStemmer() if PorterStemmer else None

# ---- VADER（可选）----
if _NLTK_OK and SentimentIntensityAnalyzer is not None:
    try:
        VADER = SentimentIntensityAnalyzer()
    except Exception:
        VADER = None
else:
    VADER = None

# ---- 月份映射 ----
_MONTH = dict(
    jan='01', feb='02', mar='03', apr='04', may='05', jun='06',
    jul='07', aug='08', sep='09', oct='10', nov='11', dec='12'
)

# ---- BODY 起点检测 ----
_BODY_OPEN = re.compile(
    r'(?is)<\s*(section|div|article)\b[^>]*\b(article-content|article-body|content-body|post-content)\b[^>]*>'
)

# ---- 工具函数 ----
def _norm(s: str) -> str:
    if not isinstance(s, str): return ""
    return ' '.join(s.lower().split())

def _slug(s: str) -> str:
    s = (s or "").strip().lower()
    s = re.sub(r'[^a-z0-9]+', '-', s).strip('-')
    return s or "unknown"

def _bucket(x, cuts):
    if x is None: return "b0"
    try:
        for i, c in enumerate(cuts):
            if x <= c: return f"b{i}"
        return f"b{len(cuts)}"
    except Exception:
        return "b0"

def _parse_wh_from_src(src: str):
    if not src: return (None, None)
    m = re.search(r'(\d{2,5})[xX](\d{2,5})', src)
    if m: return int(m.group(1)), int(m.group(2))
    m = re.search(r'[?&]w=(\d{2,5}).*?[?&]h=(\d{2,5})', src)
    if m: return int(m.group(1)), int(m.group(2))
    return (None, None)

def _img_size_bucket(w, h):
    if not w or not h: return "unk"
    area = w * h
    if area < 160*160: return "tiny"
    if area < 320*320: return "small"
    if area < 640*640: return "mid"
    if area < 1280*720: return "large"
    return "hero"

def _aspect_bucket(w, h):
    if not w or not h: return "unk"
    r = w / float(h)
    if 0.9 <= r <= 1.1: return "square"
    if r > 1.6: return "wide"
    if r < 0.6: return "tall"
    return "std"

def _safe_div(a, b, eps=1e-9):
    return float(a) / float(b + eps)

def _stem_list(text, STOP, porter):
    if not text: return []
    toks = [w for w in re.findall(r'[A-Za-z]+', text.lower()) if w not in STOP]
    return [porter.stem(w) for w in toks] if porter else toks

def _sent_split(txt: str):
    if not txt: return []
    parts = re.split(r'[.!?]+', txt)
    return [p.strip() for p in parts if p.strip()]

def _extract_canonical_domain(soup: BeautifulSoup):
    url = None
    link = soup.find('link', rel=lambda v: v and 'canonical' in v.lower())
    if link and link.get('href'): url = link['href']
    if not url:
        og = soup.find('meta', property='og:url')
        if og and og.get('content'): url = og['content']
    if not url:
        # 最后尝试 article:url / twitter:url
        for prop in ['article:url', 'twitter:url']:
            m = soup.find('meta', property=prop)
            if m and m.get('content'): url = m['content']; break
    if not url: return None, None
    try:
        p = urlparse(url)
        host = (p.netloc or '').lower()
        path = (p.path or '/')
        depth = max(0, len([seg for seg in path.split('/') if seg]))
        return host, depth
    except Exception:
        return None, None

# ---- 点击诱饵/语气/数字检测 ----
CLICKBAIT_PATTERNS = [
    (r"\bhow to\b", "howto"),
    (r"\bwhat happens when\b|\byou won't believe\b|\bcan't believe\b", "shock_believe"),
    (r"\b\d+\s+(ways|reasons)\b", "list_reason"),
    (r"\bexplained\b|\bq&a\b|\btimeline\b", "explainer"),
    (r"\blive updates?\b|\blive blog\b", "live"),
    (r"\bvs\.?\b|\bversus\b", "vs"),
    (r"\b(opinion|op\-ed|editorial)\b", "opinion"),
]

HEDGE_WORDS = {"may","might","could","possibly","reportedly","allegedly","appears","suggests","seems","likely"}
ASSERTIVE_WORDS = {"will","must","definitely","confirmed","official","guaranteed","undeniably"}
CURRENCY_SYMS = "$€£¥"
CURRENCY_WORDS = {"usd","euro","eur","dollar","dollars","pound","yen","rmb","cny"}
PERCENT_RE = re.compile(r"\b\d+(?:\.\d+)?\s*%")
MONEY_RE   = re.compile(rf"[{re.escape(CURRENCY_SYMS)}]\s*\d[\d,]*(?:\.\d+)?|\b\d[\d,]*(?:\.\d+)?\s*(?:{'|'.join(CURRENCY_WORDS)})\b", re.I)
NUMBER_RE  = re.compile(r"\b\d+(?:\.\d+)?\b")

# ---- 词袋清洗 ----
def _bow_clean(txt: str) -> str:
    toks = [w for w in re.findall(r'[A-Za-z]+', (txt or '').lower()) if w not in STOP]
    return ' '.join(porter.stem(w) for w in toks) if porter else ' '.join(toks)

# ---- Tokenizer（保留元数据规则）----
def tokenizer_stem_keepmeta(text: str, entities: set) -> list:
    toks = re.split(r'\s+', (text or '').strip()); out=[]
    for w in toks:
        if not w: continue
        if '_' in w or any(ch.isdigit() for ch in w) or (w.startswith('entity_') and w[7:] in entities):
            out.append(w)
        elif w.lower() not in STOP and re.fullmatch(r'[a-zA-Z]+', w):
            out.append(porter.stem(w.lower()) if porter else w.lower())
    return out

# =========================
#     改进版 preprocessor（对齐 Mashable 61 特征）
# =========================
def preprocessor(html: str, lda_vectorizer=None, lda_model=None, max_text_len=500):
    if not isinstance(html, str) or not html.strip():
        return "empty_content", set(), "", "", "", ""

    # --- header/body 切分 ---
    m = _BODY_OPEN.search(html)
    if m:
        header_html = html[:m.start()]
        body_html = html[m.start():]
    else:
        header_html = html
        body_html = ""

    soup = BeautifulSoup(header_html, 'html.parser')

    # --- URL/Domain/Depth ---
    domain, url_depth = _extract_canonical_domain(soup)

    # --- 标题 ---
    title_raw = None
    h1 = soup.find('h1', class_=lambda c: (isinstance(c, list) and any('title' in x for x in c)) or (isinstance(c, str) and 'title' in c)) \
         or soup.find('h1')
    if h1: title_raw = h1.get_text(' ', strip=True)
    elif soup.title: title_raw = soup.title.get_text(' ', strip=True)
    title_tokens_norm = _norm(title_raw)
    title_words = [w for w in re.findall(r"[A-Za-z]+", title_tokens_norm)]
    n_tokens_title = len(title_words)
    avg_title_tok_len = _safe_div(sum(len(w) for w in title_words), max(1, n_tokens_title))

    # --- 作者 / 频道 / 发布者 ---
    author = None
    by = soup.find(class_=lambda c: c and ('byline' in c or 'author_name' in c))
    if by: author = by.get_text(' ', strip=True)
    if not author:
        a = soup.find('a', href=re.compile(r'/author/[^/]+/?$', re.I))
        if a: author = a.get_text(' ', strip=True)
    author_slug = _slug(re.sub(r'^\s*by\s+', '', author or '', flags=re.I))

    channel = None
    art = soup.find('article')
    if art and art.has_attr('data-channel'): channel = art['data-channel']
    if not channel and art:
        cls = ' '.join(art.get('class', []))
        mch = re.search(r'\b(news|tech|world|sports?|business|entertainment|culture|life|science|lifestyle|socmed)\b', cls, re.I)
        if mch: channel = mch.group(1)
    channel_slug = _slug(channel or 'unknown')

    publisher = None
    pub = soup.find('a', href=re.compile(r'/publishers/[^/]+/?', re.I))
    if pub:
        publisher = pub.get_text(' ', strip=True) or re.sub(r'.*/publishers/([^/]+)/?.*', r'\1', pub['href'], flags=re.I)
    publisher_slug = _slug(publisher or 'unknown')

    # --- 时间 ---
    year = month = weekday = tod = season = None
    is_weekend = None
    tm = soup.find('time')
    dt = tm['datetime'] if (tm and tm.has_attr('datetime')) else (tm.get_text(' ', strip=True) if tm else None)
    if dt:
        y = re.search(r'(20\d{2}|19\d{2})', dt);  year = y.group(1) if y else None
        mo = re.search(r'-(\d{2})-', dt) or re.search(r'\b(jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec)\b', dt, re.I)
        if mo:
            mm = mo.group(1).lower() if mo.lastindex else mo.group(0).lower()
            month = _MONTH.get(mm, mm)
        wd = re.search(r'\b(mon|tue|wed|thu|fri|sat|sun)\b', dt, re.I)
        if wd:
            weekday = wd.group(1).lower(); is_weekend = weekday in ('sat','sun')
        hh = re.search(r'\b(\d{2}):(\d{2})', dt)
        if hh:
            h = int(hh.group(1))
            tod = 'morning' if 5<=h<12 else 'afternoon' if 12<=h<17 else 'evening' if 17<=h<22 else 'night'
        if month:
            m_i = int(month)
            season = 'spring' if 3<=m_i<=5 else 'summer' if 6<=m_i<=8 else 'autumn' if 9<=m_i<=11 else 'winter'

    # weekday flags（模仿 weekday_is_*）
    weekday_flags = []
    if weekday:
        wd_map = dict(mon='monday', tue='tuesday', wed='wednesday', thu='thursday', fri='friday', sat='saturday', sun='sunday')
        for k,v in wd_map.items():
            weekday_flags.append(f"weekday_is_{v}" if weekday==k else f"weekday_is_{v}=0")

    # --- 媒体元素 + 链接 ---
    imgs = soup.find_all('img'); img_count = len(imgs); has_image = img_count > 0
    leadimg = soup.find(attrs={'data-fragment':'lead-image'}) is not None
    max_w=max_h=None
    for im in imgs:
        w,h = _parse_wh_from_src(im.get('src',''))
        if w and h and ((not max_w) or (w*h) > ((max_w or 0)*(max_h or 0))):
            max_w,max_h = w,h
    img_size_bucket = _img_size_bucket(max_w, max_h)
    img_aspect_bucket = _aspect_bucket(max_w, max_h)
    videos = soup.find_all('video'); iframes = soup.find_all('iframe')
    # iframe 中判定第三方视频
    video_iframes = [fr for fr in iframes if re.search(r'(youtube|vimeo|dailymotion)', (fr.get('src') or ''), re.I)]
    num_videos = len(videos) + len(video_iframes)
    audio = soup.find_all('audio'); has_audio = len(audio)>0

    a_tags_header = soup.find_all('a')
    link_count = len(a_tags_header)
    link_bucket = _bucket(link_count, [0,1,3,6,10])

    domain_self_cnt = 0
    if domain:
        for a in a_tags_header:
            href = (a.get('href') or '').strip()
            if href.startswith('http'):
                try:
                    host = urlparse(href).netloc.lower()
                    if host and domain and (host.endswith(domain) or domain.endswith(host)):
                        domain_self_cnt += 1
                except Exception:
                    pass
    num_self_hrefs = domain_self_cnt
    self_ref_ratio = _safe_div(num_self_hrefs, max(1, link_count))

    authoritative_domains = ['.edu','.gov','.org']
    authoritative_links = sum(1 for a in a_tags_header if any(d in (a.get('href') or '').lower() for d in authoritative_domains))
    authoritative_link_bucket = _bucket(authoritative_links, [0,1,3,5])

    # --- 标题形态 + 点击诱饵 ---
    raw = title_raw or ''
    raw_l = raw.lower()
    title_has_num = bool(re.search(r'\d', raw))
    title_has_year = bool(re.search(r'\b(19|20)\d{2}\b', raw))
    title_has_q = '?' in raw
    title_has_exclaim = '!' in raw
    title_has_colon = ':' in raw
    is_listicle = bool(re.match(r'^\s*\d+', raw))
    upper_ratio = (sum(ch.isupper() for ch in raw) / max(1, sum(ch.isalpha() for ch in raw)))
    upper_bucket = 'low' if upper_ratio < 0.15 else 'mid' if upper_ratio < 0.4 else 'high'
    title_word_len = n_tokens_title
    tw_bucket = _bucket(title_word_len, [0,4,8,12,20])
    title_char_len = len(re.sub(r'\s+','',raw)); tc_bucket = _bucket(title_char_len, [0,30,60,90,140])

    title_clickbait_tags = []
    for pat, tag in CLICKBAIT_PATTERNS:
        if re.search(pat, raw_l):
            title_clickbait_tags.append(f"title_clickbait_{tag}")
    if not title_clickbait_tags:
        title_clickbait_tags.append("title_clickbait_none")

    # --- header 文本内容 + 情感 ---
    text_content = ' '.join(soup.get_text().lower().split()[:max_text_len])

    # VADER 标题/头部情感
    if VADER is not None:
        vader_header = VADER.polarity_scores(text_content)
        vader_title  = VADER.polarity_scores(title_raw or '')
        header_compound = vader_header['compound']
        header_pos, header_neg, header_neu = vader_header['pos'], vader_header['neg'], vader_header['neu']
        title_compound  = vader_title['compound']
        title_pos, title_neg, title_neu = vader_title['pos'], vader_title['neg'], vader_title['neu']
    else:
        header_compound = title_compound = 0.0
        header_pos = header_neg = header_neu = 0.0
        title_pos  = title_neg  = title_neu  = 0.0

    def _pol_bucket(x):
        return _bucket(x, [0.0, 0.1, 0.2, 0.4, 0.6, 0.8])

    # 词性标注（安全降级）
    if _NLTK_OK and word_tokenize and pos_tag:
        tokens = word_tokenize(text_content)
        try:
            tagged = pos_tag(tokens)
        except Exception:
            tagged = [(w,'NN') for w in tokens]
    else:
        tokens = text_content.split()
        tagged = [(w,'NN') for w in tokens]
    nouns = [w for w,pos_ in tagged if pos_.startswith('NN') and w.lower() not in STOP]
    noun_count_bucket = _bucket(len(nouns), [0,5,10,20,50])

    # --- NER（细类计数，可关） ---
    entities = set()
    person_cnt = org_cnt = gpe_cnt = 0
    if not FAST_NO_NER and _NLTK_OK and ne_chunk:
        try:
            chunked = ne_chunk(tagged)
            for ch in chunked:
                if hasattr(ch,'label') and ch.label() in ['PERSON','ORGANIZATION','GPE']:
                    ent = '_'.join(c[0].lower() for c in ch)
                    entities.add(f'entity_{ent}')
                    if ch.label() == 'PERSON': person_cnt += 1
                    elif ch.label() == 'ORGANIZATION': org_cnt += 1
                    elif ch.label() == 'GPE': gpe_cnt += 1
        except Exception:
            pass
    entity_count_bucket = _bucket(len(entities), [0,1,3,5])

    # --- LDA：多 topic 分布（模仿 LDA_00..LDA_04） ---
    lda_tokens = []
    if (lda_vectorizer is not None) and (lda_model is not None):
        try:
            # 优先用正文，否则用 header
            text_for_lda = text_content
            if body_html:
                soup_body_tmp = BeautifulSoup(body_html, 'html.parser')
                text_for_lda = ' '.join(soup_body_tmp.get_text().lower().split())
            X_ = lda_vectorizer.transform([text_for_lda] if text_for_lda else [''])
            topic_dist = lda_model.transform(X_)
            if topic_dist.ndim == 2 and topic_dist.shape[1] > 0:
                K = min(topic_dist.shape[1], 5)  # 最多取5个主题，对齐 LDA_00..LDA_04
                for i in range(K):
                    s = float(topic_dist[0][i])
                    lda_tokens.append(f"LDA_{i:02d}_b{_bucket(s,[0,0.2,0.4,0.6,0.8])}")
                dom = int(np.argmax(topic_dist[0]))
                lda_tokens.append(f"LDA_dom_t{dom}")
            else:
                lda_tokens.append('LDA_00_unk')
        except Exception:
            lda_tokens.append('LDA_00_unk')
    else:
        lda_tokens.append('LDA_00_unk')

    # --- 可读性（可关） ---
    if _HAS_TEXTSTAT:
        try:
            readability_score = flesch_reading_ease(text_content) if text_content else 0
        except Exception:
            readability_score = 0
    else:
        readability_score = 50
    readability_bucket = ('very_easy' if readability_score>80 else
                          'easy' if readability_score>60 else
                          'standard' if readability_score>50 else
                          'difficult' if readability_score>30 else 'very_difficult')

    # --- header 结构统计 ---
    div_count = len(soup.find_all('div'))
    section_count = len(soup.find_all('section'))
    list_count = len(soup.find_all(['ul','ol']))
    div_count_bucket = _bucket(div_count, [0,5,10,20,50])
    section_count_bucket = _bucket(section_count, [0,1,3,5])
    list_count_bucket = _bucket(list_count, [0,1,3,5])
    header_word_count_bucket = _bucket(len(text_content.split()), [0,50,100,200,500])

    # ====== header 特征拼装 ======
    feats = []
    feats += [
        f'author_{author_slug or "unknown"}',
        f'channel_{channel_slug}', f'publisher_{publisher_slug}',
        f'year_{year or "unk"}', f'month_{month or "unk"}',
        f'weekday_{weekday or "unk"}', f'tod_{tod or "unk"}', f'season_{season or "unk"}',
        'weekend' if is_weekend else 'weekday' if is_weekend is not None else 'weekend_unk',
    ]
    # weekday flags
    feats += weekday_flags

    # Mashable 61 特征的核心部分
    feats += [
        # 基础计数特征
        f'n_tokens_title_{_bucket(n_tokens_title, [0,4,8,12,20,30])}',
        f'num_hrefs_{_bucket(link_count, [0,1,3,6,10,20,40])}',
        f'num_self_hrefs_{_bucket(num_self_hrefs, [0,1,2,3,5,10])}',
        f'num_imgs_{_bucket(img_count, [0,1,2,4,8,16])}',
        f'num_videos_{_bucket(num_videos, [0,1,2,3,5])}',
        f'average_token_length_{_bucket(avg_title_tok_len, [0,3,5,7,9])}',
        f'num_keywords_{_bucket(len(kws) if 'kws' in locals() else 0, [0,1,3,5,10,20])}',
        
        # 媒体特征
        'has_image' if has_image else 'no_image',
        'has_leadimg' if leadimg else 'no_leadimg',
        f'imgsize_{img_size_bucket}', f'imgaspect_{img_aspect_bucket}',
        'has_video' if num_videos>0 else 'no_video',
        'has_audio' if has_audio else 'no_audio',
    ]

    # 频道标志（data_channel_is_*）
    channel_map = {
        'lifestyle': 'lifestyle', 'life': 'lifestyle',
        'entertainment': 'entertainment', 'culture': 'entertainment',
        'business': 'bus', 'news': 'world', 'world':'world',
        'socmed':'socmed', 'social':'socmed',
        'tech':'tech', 'science':'tech', 'sports':'sports', 'sport':'sports'
    }
    cat = channel_map.get(channel_slug, None)
    for key in ['lifestyle','entertainment','bus','socmed','tech','world','sports']:
        feats.append(f'data_channel_is_{key}' if cat==key else f'data_channel_is_{key}=0')

    # 标题特征
    feats += [
        'is_listicle' if is_listicle else 'not_listicle',
        'title_has_num' if title_has_num else 'title_no_num',
        'title_has_year' if title_has_year else 'title_no_year',
        'title_has_q' if title_has_q else 'title_no_q',
        'title_has_exclaim' if title_has_exclaim else 'title_no_exclaim',
        'title_has_colon' if title_has_colon else 'title_no_colon',
        f'title_len_word_{tw_bucket}', f'title_len_char_{tc_bucket}', f'title_upper_{upper_bucket}',
    ]

    # 标题点击诱饵/风格
    feats += (title_clickbait_tags or ["title_clickbait_none"])  

    # NER / 结构 / 可读性 / 名词数
    feats += [
        f'ner_person_{_bucket(person_cnt,[0,1,3,6])}',
        f'ner_org_{_bucket(org_cnt,[0,1,3,6])}',
        f'ner_gpe_{_bucket(gpe_cnt,[0,1,3,6])}',
        f'div_count_{div_count_bucket}', f'section_count_{section_count_bucket}',
        f'list_count_{list_count_bucket}', f'readability_{readability_bucket}',
        f'header_word_count_{header_word_count_bucket}',
        f'noun_count_{noun_count_bucket}',
    ]

    # 关键词（meta keywords / article:tag）
    kw_tokens = []
    meta_kw = soup.find('meta', attrs={'name':'keywords'})
    kws = []
    if meta_kw and meta_kw.get('content'):
        kws += [k.strip().lower() for k in meta_kw['content'].split(',') if k.strip()]
    for meta_tag in soup.find_all('meta', attrs={'property':'article:tag'}):
        if meta_tag.get('content'): kws.append(meta_tag['content'].strip().lower())
    kws = [k for k in kws if k]
    num_keywords = len(kws)
    
    # 关键词统计特征（模拟 kw_min_min, kw_max_min 等）
    if kws:
        kw_lengths = [len(kw) for kw in kws]
        feats += [
            f'kw_min_min_{_bucket(min(kw_lengths) if kw_lengths else 0, [0,3,5,7,10])}',
            f'kw_max_min_{_bucket(max(kw_lengths) if kw_lengths else 0, [0,5,10,15,20])}',
            f'kw_avg_min_{_bucket(np.mean(kw_lengths) if kw_lengths else 0, [0,3,6,9,12])}',
        ]
    else:
        feats += ['kw_min_min_0', 'kw_max_min_0', 'kw_avg_min_0']

    kw_set = set(kws)
    title_set = set(w.lower() for w in title_words)
    kw_overlap_title = _safe_div(len(kw_set & title_set), max(1, len(kw_set))) if kw_set else 0.0
    feats += [f'keyword_overlap_title_{_bucket(int(100*kw_overlap_title), [0,10,25,50,75,90])}']

    # URL 特征
    if domain:
        feats += [f'url_domain_{_slug(domain)}']
    if url_depth is not None:
        feats += [f'url_depth_{_bucket(url_depth, [0,1,2,3,5,8])}']

    # 情感特征（61-like）
    feats += [
        f'global_subjectivity_{_bucket(header_neu, [0,0.2,0.4,0.6,0.8])}',  # 用中性词比例代理主观性
        f'global_sentiment_polarity_{_pol_bucket(header_compound)}',
        f'global_rate_positive_words_{_pol_bucket(header_pos)}',
        f'global_rate_negative_words_{_pol_bucket(header_neg)}',
        f'rate_positive_words_{_pol_bucket(_safe_div(header_pos, header_pos+header_neg))}',
        f'rate_negative_words_{_pol_bucket(_safe_div(header_neg, header_pos+header_neg))}',
        f'title_subjectivity_{_pol_bucket(title_neu)}',
        f'title_sentiment_polarity_{_pol_bucket(title_compound)}',
        f'abs_title_subjectivity_{_pol_bucket(abs(title_neu))}',
        f'abs_title_sentiment_polarity_{_pol_bucket(abs(title_compound))}',
    ]

    # 将 LDA token 也加入 header feats
    feats += lda_tokens

    header_bow_text = _bow_clean(text_content)

    # ====== 正文特征 ======
    body_feats = []
    body_bow_text = ""
    title_stems = set(_stem_list(title_raw or "", STOP, porter))

    if body_html:
        soup_body = BeautifulSoup(body_html, 'html.parser')
        body_text = ' '.join(soup_body.get_text().lower().split())
        body_bow_text = _bow_clean(body_text)

        # 基本规模
        body_words = [w for w in re.findall(r"[A-Za-z]+", body_text)]
        body_len = len(body_words)
        
        # Mashable 61 特征：正文相关
        uniq = len(set(body_words))
        n_unique_tokens = _safe_div(uniq, max(1, body_len))
        nostop_words = [w for w in body_words if w.lower() not in STOP]
        n_non_stop_words = _safe_div(len(nostop_words), max(1, body_len))
        n_non_stop_unique_tokens = _safe_div(len(set(w.lower() for w in nostop_words)), max(1, body_len))
        avg_token_length = _safe_div(sum(len(w) for w in body_words), max(1, body_len))

        body_feats += [
            f'n_tokens_content_{_bucket(body_len, [0,100,300,600,1000,2000,4000])}',
            f'n_unique_tokens_{_bucket(int(100*n_unique_tokens), [0,20,30,40,50,60,70,80])}',
            f'n_non_stop_words_{_bucket(int(100*n_non_stop_words), [0,20,40,60,75,90])}',
            f'n_non_stop_unique_tokens_{_bucket(int(100*n_non_stop_unique_tokens), [0,10,20,30,40,50,60])}',
            f'average_token_length_{_bucket(avg_token_length, [0,3,4,5,6,7,8])}',
        ]

        # 段落/引用/代码/小标题
        p_count = len(soup_body.find_all('p'))
        bq_count = len(soup_body.find_all('blockquote'))
        code_count = len(soup_body.find_all(['pre','code']))
        h2h3_count = len(soup_body.find_all(['h2','h3']))
        body_feats += [
            f'body_p_{_bucket(p_count,[0,5,15,30,50])}',
            f'body_bq_{_bucket(bq_count,[0,1,3,5])}',
            'body_has_code' if code_count>0 else 'body_no_code',
            f'body_h2h3_{_bucket(h2h3_count,[0,1,3,6,10])}',
        ]

        # 外链比例/权威外链（正文）
        a_tags = soup_body.find_all('a')
        ext_links = [a for a in a_tags if re.match(r'^https?://', (a.get('href') or ''), re.I)]
        ext_ratio = _safe_div(len(ext_links), max(1,len(a_tags)))
        body_feats += [
            f'links_ext_ratio_{_bucket(int(100*ext_ratio), [0,10,30,60,90])}',
            f'links_ext_cnt_{_bucket(len(ext_links), [0,1,3,6,10,20])}',
        ]
        authoritative_domains = ['.edu','.gov','.org']
        auth_links_body = sum(1 for a in ext_links if any(d in (a.get('href') or '').lower() for d in authoritative_domains))
        body_feats.append(f'links_auth_cnt_{_bucket(auth_links_body,[0,1,3,5,10])}')

        # 引号/标点/句长
        quotes_cnt = body_text.count('"') + body_text.count('"') + body_text.count('"') + body_text.count("'") + body_text.count("'")
        exclaim_cnt = body_text.count('!')
        qmark_cnt = body_text.count('?')
        sents = _sent_split(body_text)
        avg_sl = _safe_div(sum(len(s.split()) for s in sents), max(1,len(sents)))
        body_feats += [
            f'quotes_{_bucket(quotes_cnt,[0,1,3,6,10])}',
            f'body_exclaim_{_bucket(exclaim_cnt,[0,1,2,4,8])}',
            f'body_qmark_{_bucket(qmark_cnt,[0,1,2,4,8])}',
            f'avg_sentence_len_{_bucket(int(avg_sl), [0,10,15,20,30,40])}'
        ]

        # 词汇多样性/阅读时长
        lex_div = _safe_div(uniq, max(1,body_len))
        read_min = int(round(body_len / 200.0))  # 200 wpm 粗估
        body_feats += [
            f'lexdiv_{_bucket(int(100*lex_div), [0,20,30,40,50,60])}',
            f'readtime_min_{_bucket(read_min,[0,1,2,4,6,10])}'
        ]

        # 金额/百分比/数字密度
        money_cnt = len(MONEY_RE.findall(body_text))
        pct_cnt = len(PERCENT_RE.findall(body_text))
        num_cnt = len(NUMBER_RE.findall(body_text))
        body_feats += [
            f'money_{_bucket(money_cnt,[0,1,2,5,10])}',
            f'percent_{_bucket(pct_cnt,[0,1,2,5,10])}',
            f'numdens_{_bucket(int(1000*_safe_div(num_cnt, max(1,body_len))), [0,5,10,20,40,80])}'
        ]

        # 情感极性统计（正文）
        if VADER is not None:
            vd_body = VADER.polarity_scores(body_text)
            # 句子级情感统计
            if sents:
                sent_scores = [VADER.polarity_scores(s)['compound'] for s in sents]
                pos_scores = [s for s in sent_scores if s > 0]
                neg_scores = [s for s in sent_scores if s < 0]
                
                if pos_scores:
                    body_feats += [
                        f'avg_positive_polarity_{_pol_bucket(np.mean(pos_scores))}',
                        f'min_positive_polarity_{_pol_bucket(min(pos_scores))}',
                        f'max_positive_polarity_{_pol_bucket(max(pos_scores))}',
                    ]
                else:
                    body_feats += ['avg_positive_polarity_0', 'min_positive_polarity_0', 'max_positive_polarity_0']
                    
                if neg_scores:
                    body_feats += [
                        f'avg_negative_polarity_{_pol_bucket(abs(np.mean(neg_scores)))}',
                        f'min_negative_polarity_{_pol_bucket(abs(min(neg_scores)))}',
                        f'max_negative_polarity_{_pol_bucket(abs(max(neg_scores)))}',
                    ]
                else:
                    body_feats += ['avg_negative_polarity_0', 'min_negative_polarity_0', 'max_negative_polarity_0']
            else:
                body_feats += ['avg_positive_polarity_0', 'min_positive_polarity_0', 'max_positive_polarity_0',
                              'avg_negative_polarity_0', 'min_negative_polarity_0', 'max_negative_polarity_0']

        # 标题-正文一致性
        body_stems = set(_stem_list(body_text, STOP, porter))
        jacc = _safe_div(len(title_stems & body_stems), max(1, len(title_stems | body_stems)))
        title_ents_like = set(re.findall(r'[A-Za-z][A-Za-z0-9_]+', (title_tokens_norm or '')))
        ent_hit = _safe_div(sum(1 for t in title_ents_like if (t or "").lower() in body_text), max(1, len(title_ents_like)))
        body_feats += [
            f'align_jacc_{_bucket(int(100*jacc), [0,10,20,35,50,70])}',
            f'align_ent_{_bucket(int(100*ent_hit), [0,10,25,50,75,90])}'
        ]

        # 图片密度
        body_imgs = soup_body.find_all('img')
        img_density = _safe_div(len(body_imgs), max(1, body_len/500.0))
        body_feats.append(f'imgdens_{_bucket(int(10*img_density), [0,1,2,3,5,8])}')

        # 关键词与正文重合
        if num_keywords>0:
            kw_hit_body = _safe_div(len(kw_set & set(w.lower() for w in body_words)), max(1, len(kw_set)))
            body_feats.append(f'keyword_overlap_body_{_bucket(int(100*kw_hit_body), [0,10,25,50,75,90])}')

    else:
        body_feats.append('no_body')
        # 如果没有正文，设置默认值
        body_feats += [
            'n_tokens_content_0', 'n_unique_tokens_0', 'n_non_stop_words_0',
            'n_non_stop_unique_tokens_0', 'average_token_length_0'
        ]

    # === 将标准化后的标题返回 ===
    return _norm(title_tokens_norm), entities, ' '.join(feats), _bow_clean(text_content), ' '.join(body_feats), body_bow_text

# =========================
#  向量器 / Hasher 定义 - 更新前缀映射以包含新特征
# =========================

# 词袋向量（标题 / header / 实体 / 正文）
title_vec   = HashingVectorizer(n_features=2**20, alternate_sign=False, ngram_range=(1,2), token_pattern=r'(?u)\b\w+\b')
header_vec  = HashingVectorizer(n_features=2**18, alternate_sign=True,  ngram_range=(1,2), token_pattern=r'(?u)\b\w+\b')
entities_vec= HashingVectorizer(n_features=2**12, alternate_sign=False, token_pattern=r'(?u)\b\w+\b')
body_vec    = HashingVectorizer(n_features=2**20, alternate_sign=False, ngram_range=(1,2), token_pattern=r'(?u)\b\w+\b')

make_hasher = lambda n: FeatureHasher(n_features=2**n, input_type='string', alternate_sign=False)

# 原有组 + 新增组
author_h, channel_h, publisher_h = make_hasher(10), make_hasher(10), make_hasher(10)
time_h, media_h, titleshape_h   = make_hasher(10), make_hasher(12), make_hasher(10)
social_h, content_h, structure_h= make_hasher(14), make_hasher(14), make_hasher(14)
length_h, extra_h               = make_hasher(12), make_hasher(14)
bodyfeats_h                     = make_hasher(12)

# Mashable 61 特征专用分组
mashable_basic_h = make_hasher(12)   # n_tokens_title, num_hrefs 等基础特征
mashable_channel_h = make_hasher(8)  # data_channel_is_*
mashable_kw_h = make_hasher(11)      # kw_* 关键词特征
mashable_sentiment_h = make_hasher(12) # 情感相关特征
mashable_lda_h = make_hasher(10)     # LDA 主题特征
mashable_content_h = make_hasher(12) # n_tokens_content 等正文特征

# =========================
#  前缀映射（分组哈希）- 更新以包含 Mashable 特征
# =========================
PREFIX_MAP = {
    'author_':'author','channel_':'channel','publisher_':'publisher',
    'year_':'time','month_':'time','weekday_':'time','tod_':'time','season_':'time',
    'weekend':'time','weekday':'time',
    'weekday_is_':'time',

    'has_image':'media','no_image':'media','imgcnt_':'media','has_leadimg':'media','no_leadimg':'media',
    'imgsize_':'media','imgaspect_':'media','has_video':'media','no_video':'media','has_audio':'media','no_audio':'media',

    # Mashable 61 基础特征
    'n_tokens_title_':'mashable_basic',
    'num_hrefs_':'mashable_basic','num_self_hrefs_':'mashable_basic','num_imgs_':'mashable_basic','num_videos_':'mashable_basic',
    'average_token_length_':'mashable_basic','num_keywords_':'mashable_basic',
    
    # Mashable 频道标志
    'data_channel_is_':'mashable_channel',
    
    # Mashable 关键词特征
    'kw_min_min_':'mashable_kw','kw_max_min_':'mashable_kw','kw_avg_min_':'mashable_kw',
    
    # Mashable 情感特征
    'global_subjectivity_':'mashable_sentiment','global_sentiment_polarity_':'mashable_sentiment',
    'global_rate_positive_words_':'mashable_sentiment','global_rate_negative_words_':'mashable_sentiment',
    'rate_positive_words_':'mashable_sentiment','rate_negative_words_':'mashable_sentiment',
    'title_subjectivity_':'mashable_sentiment','title_sentiment_polarity_':'mashable_sentiment',
    'abs_title_subjectivity_':'mashable_sentiment','abs_title_sentiment_polarity_':'mashable_sentiment',
    'avg_positive_polarity_':'mashable_sentiment','min_positive_polarity_':'mashable_sentiment','max_positive_polarity_':'mashable_sentiment',
    'avg_negative_polarity_':'mashable_sentiment','min_negative_polarity_':'mashable_sentiment','max_negative_polarity_':'mashable_sentiment',
    
    # Mashable LDA
    'LDA_':'mashable_lda',
    
    # Mashable 正文特征
    'n_tokens_content_':'mashable_content','n_unique_tokens_':'mashable_content',
    'n_non_stop_words_':'mashable_content','n_non_stop_unique_tokens_':'mashable_content',
    
    # 原有其他特征
    'linkcnt_':'media','authoritative_links_':'media',
    'num_hrefs_b':'links','num_self_hrefs_b':'links','self_ref_ratio_b':'links',
    'is_listicle':'titleshape','not_listicle':'titleshape','title_has_':'titleshape',
    'title_len_word_':'titleshape','title_len_char_':'titleshape','title_upper_':'titleshape',
    'title_clickbait_':'style',
    'social_buttons_':'social','comment_sections_':'social','share_count_':'social',
    'urgency_indicators_':'content','question_words_':'content','noun_count_':'content','cta_count_':'content',
    'div_count_':'structure','section_count_':'structure','list_count_':'structure','readability_':'structure',
    'header_word_count_':'length',
    'entity_count_':'extra','topic_':'extra','sentiment_':'extra','positive_words_':'extra','negative_words_':'extra',
    'ner_person_':'extra','ner_org_':'extra','ner_gpe_':'extra',
    'body_len_':'length',
    'body_p_':'sections','body_bq_':'sections','body_has_code':'sections','body_no_code':'sections','body_h2h3_':'sections',
    'links_ext_ratio_':'links','links_ext_cnt_':'links','links_auth_cnt_':'links',
    'quotes_':'quotes','body_exclaim_':'quotes','body_qmark_':'quotes','avg_sentence_len_':'quotes',
    'lexdiv_':'lexical','readtime_min_':'readtime',
    'money_':'numbers','percent_':'numbers','numdens_':'numbers',
    'subj_adj_':'subjectivity','subj_adv_':'subjectivity','pronouns_':'subjectivity',
    'hedge_':'subjectivity','assertive_':'subjectivity',
    'align_jacc_':'alignment','align_ent_':'alignment',
    'imgdens_':'media',
    'keyword_overlap_title_':'keywords','keyword_overlap_body_':'keywords',
    'url_domain_':'url','url_depth_':'url',
}

def _split_feat_tokens(feats_string: str):
    buckets = {k: [] for k in [
        'author','channel','publisher','time','media','titleshape','style',
        'social','content','structure','length','extra',
        'links','numbers','subjectivity','quotes','sections','readtime','lexical','alignment',
        'keywords','url',
        # Mashable 61 特征分组
        'mashable_basic','mashable_channel','mashable_kw','mashable_sentiment','mashable_lda','mashable_content'
    ]}
    if not feats_string:
        for k in buckets: buckets[k] = [f'{k}=_none']
        return buckets

    for tok in feats_string.split():
        placed = False
        for pref, grp in PREFIX_MAP.items():
            if tok.startswith(pref):
                buckets[grp].append(tok); placed=True; break
        if not placed:
            buckets['extra'].append(tok)

    for k,v in buckets.items():
        if not v:
            buckets[k] = [f'{k}=_none']
    return buckets

# =========================
#  featurize_split（兼容新增 Mashable 分组）
# =========================
def featurize_split(html_series: pd.Series, lda_vectorizer=None, lda_model=None, n_jobs=1) -> sp.csr_matrix:
    rows = html_series.astype(str).tolist()

    if n_jobs is None or n_jobs == 1:
        processed_data = [preprocessor(h, lda_vectorizer, lda_model) for h in rows]
    else:
        processed_data = Parallel(n_jobs=n_jobs, backend="loky", prefer="processes")(
            delayed(preprocessor)(h, lda_vectorizer, lda_model) for h in rows
        )

    # 收集器
    titles, header_bows, entity_texts = [], [], []
    author_tokens, channel_tokens, publisher_tokens = [], [], []
    time_tokens, media_tokens, titleshape_tokens = [], [], []
    social_tokens, content_tokens, structure_tokens = [], [], []
    length_tokens, extra_tokens = [], []
    body_feats_tokens, body_bows = [], []
    
    # Mashable 61 特征收集器
    mashable_basic_tokens, mashable_channel_tokens, mashable_kw_tokens = [], [], []
    mashable_sentiment_tokens, mashable_lda_tokens, mashable_content_tokens = [], [], []

    for title, ents, header_feats, header_bow, body_feats, body_bow in processed_data:
        titles.append(' '.join(tokenizer_stem_keepmeta(title, set(ents))) if title else '')
        header_bows.append(header_bow or '')
        entity_texts.append(' '.join(sorted(ents)) if ents else '')

        # header -> 分组
        b = _split_feat_tokens(header_feats)
        author_tokens.append(b['author']);    channel_tokens.append(b['channel']);   publisher_tokens.append(b['publisher'])
        time_tokens.append(b['time']);        media_tokens.append(b['media']);       titleshape_tokens.append(b['titleshape'])
        social_tokens.append(b['social']);    content_tokens.append(b['content']);   structure_tokens.append(b['structure'])
        length_tokens.append(b['length']);    extra_tokens.append(b['extra'])
        
        # Mashable 61 特征
        mashable_basic_tokens.append(b.get('mashable_basic', []))
        mashable_channel_tokens.append(b.get('mashable_channel', []))
        mashable_kw_tokens.append(b.get('mashable_kw', []))
        mashable_sentiment_tokens.append(b.get('mashable_sentiment', []))
        mashable_lda_tokens.append(b.get('mashable_lda', []))

        # body -> 分组
        bf = _split_feat_tokens(body_feats or '')
        body_feats_tokens.append(body_feats.split() if body_feats else ['no_body'])
        body_bows.append(body_bow or '')

        # Mashable 正文特征
        mashable_content_tokens.append(bf.get('mashable_content', []))
        mashable_sentiment_tokens[-1] += bf.get('mashable_sentiment', [])

    # 编码
    X_title      = title_vec.transform(titles)
    X_header     = header_vec.transform(header_bows)
    X_entities   = entities_vec.transform(entity_texts)

    X_author     = author_h.transform(author_tokens)
    X_channel    = channel_h.transform(channel_tokens)
    X_publisher  = publisher_h.transform(publisher_tokens)
    X_time       = time_h.transform(time_tokens)
    X_media      = media_h.transform(media_tokens)
    X_titleshape = titleshape_h.transform(titleshape_tokens)
    X_social     = social_h.transform(social_tokens)
    X_content    = content_h.transform(content_tokens)
    X_structure  = structure_h.transform(structure_tokens)
    X_length     = length_h.transform(length_tokens)
    X_extra      = extra_h.transform(extra_tokens)

    # Mashable 61 特征编码
    X_mashable_basic = mashable_basic_h.transform(mashable_basic_tokens)
    X_mashable_channel = mashable_channel_h.transform(mashable_channel_tokens)
    X_mashable_kw = mashable_kw_h.transform(mashable_kw_tokens)
    X_mashable_sentiment = mashable_sentiment_h.transform(mashable_sentiment_tokens)
    X_mashable_lda = mashable_lda_h.transform(mashable_lda_tokens)
    X_mashable_content = mashable_content_h.transform(mashable_content_tokens)

    # 正文块
    X_body_feats   = bodyfeats_h.transform(body_feats_tokens)
    X_body_bow     = body_vec.transform(body_bows)

    # 拼接 - 将 Mashable 特征集成到现有管道中
    X = sp.hstack([
        # 词袋块
        X_title, X_header, X_entities,
        # header 类别块
        X_author, X_channel, X_publisher, X_time,
        X_media, X_titleshape, X_social, X_content, X_structure, X_length, X_extra,
        # Mashable 61 特征块
        X_mashable_basic, X_mashable_channel, X_mashable_kw, 
        X_mashable_sentiment, X_mashable_lda, X_mashable_content,
        # body 类别块 + 词袋
        X_body_feats, X_body_bow
    ], format='csr')

    return X

把驗證集轉成特徵，後面評估直接用

In [ ]:
# 一次性把驗證集轉成特徵，後面評估直接用
X_val = featurize_split(val_df['Page content'].astype(str), lda_vectorizer, lda_model, n_jobs=1)
y_val = val_df['Popularity'].values
sp.save_npz(f'{OUT_DIR}/X_val_split.npz', X_val)
np.save(f'{OUT_DIR}/y_val.npy', y_val)
X_val.shape, np.bincount(y_val)


ValueError: too many values to unpack (expected 6)

In [87]:
# 先导入所有需要的库
import os
from pathlib import Path
import pickle as pkl
import copy
from math import ceil
import numpy as np
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import roc_auc_score

# ---------- 路径与原子写入工具 ----------
OUT_DIR = './output'
Path(OUT_DIR).mkdir(parents=True, exist_ok=True)

BEST_PATH  = f'{OUT_DIR}/clf-sgd-best.pkl'   # 历史最优（被提升时覆盖）
LAST_PATH  = f'{OUT_DIR}/clf-sgd-last.pkl'   # 每个 epoch 结束后覆盖
FINAL_PATH = f'{OUT_DIR}/clf-sgd-final.pkl'  # 全部训练结束后保存一次
LDA_PATH   = f'{OUT_DIR}/lda.pkl'            # LDA 组件（若有）

def dump_atomic(obj, path):
    tmp = path + '.tmp'
    with open(tmp, 'wb') as f:
        pkl.dump(obj, f, protocol=pkl.HIGHEST_PROTOCOL)
    os.replace(tmp, path)  # 原子替换，避免损坏

# ---------- 训练生成器 ----------
BATCH_SIZE = 1000  # 可调大些提高吞吐
def train_streamer(df, batch_size):
    for i in range(0, len(df), batch_size):
        yield df.iloc[i:i+batch_size]

# ---------- 模型与训练超参 ----------
classes = np.array([0, 1])
clf = SGDClassifier(
    # 损失函数
    loss="squared_hinge",  # 可选: 'hinge', 'log_loss', 'modified_huber', 'squared_hinge', 
                          # 'perceptron', 'squared_error', 'huber', 'epsilon_insensitive', 
                          # 'squared_epsilon_insensitive'
    
    # 正则化
    penalty="elasticnet",  # 可选: 'l2', 'l1', 'elasticnet', None
    alpha=1e-6,           # 正则化强度，通常范围: 1e-7 到 1e-2
    l1_ratio=0.15,        # Elastic Net 混合参数，范围: 0 到 1
    
    # 学习率调度
    learning_rate="constant",  # 可选: 'constant', 'optimal', 'invscaling', 'adaptive'
    eta0=1e-3,                 # 初始学习率
    power_t=0.25,              # inverse scaling 的指数 [默认 0.25]
    
    # 训练控制
    max_iter=1000,             # 最大迭代次数，建议: 100-5000
    tol=1e-4,                  # 停止容忍度，建议: 1e-4 到 1e-2
    shuffle=True,              # 是否每个epoch后打乱数据
    early_stopping=False,      # 是否使用早停
    validation_fraction=0.1,   # 早停验证集比例
    n_iter_no_change=5,        # 早停等待轮数
    
    # 模型配置
    fit_intercept=True,        # 是否计算截距
    average=False,             # 是否使用平均SGD
    n_jobs=None,               # 并行数 (-1 使用所有CPU)
    random_state=28,
    
    # 类别权重
    class_weight=None,         # 可选: 'balanced', dict, 或 None
    
    # 其他
    verbose=0,                 # 详细程度
    warm_start=False,          # 是否热启动
)

EPOCHS = 50
EVAL_EVERY = 1  # 每步都评估（你原来的逻辑）
best_val_auc = -np.inf
best_clf = None
best_epoch = 0

train_auc_hist, val_auc_hist = [], []

# ---------- 多 epoch 训练 ----------
for epoch in range(1, EPOCHS + 1):
    print(f"\n=== Epoch {epoch}/{EPOCHS} ===")

    # 每个 epoch 打乱
    epoch_train_df = train_df.sample(frac=1.0, random_state=28 + epoch).reset_index(drop=True)
    stream = train_streamer(epoch_train_df, BATCH_SIZE)
    iters = ceil(len(epoch_train_df) / BATCH_SIZE)

    epoch_train_aucs = []

    for i, batch in enumerate(stream, start=1):
        Xtr = featurize_split(batch['Page content'].astype(str), lda_vectorizer, lda_model, n_jobs=1)
        ytr = batch['Popularity'].values
        if epoch == 1 and i == 1:
            clf.partial_fit(Xtr, ytr, classes=classes)
        else:
            clf.partial_fit(Xtr, ytr)

        # 批内 AUC
        if np.unique(ytr).size == 2:
            tr_auc = roc_auc_score(ytr, clf.decision_function(Xtr))
        else:
            tr_auc = np.nan
        epoch_train_aucs.append(tr_auc)
        train_auc_hist.append(tr_auc)

        # 週期性评估（用预计算 X_val）
        if i % EVAL_EVERY == 0 or i == iters:
            va_auc = roc_auc_score(y_val, clf.decision_function(X_val))
            val_auc_hist.append(va_auc)
            print(f"Epoch {epoch} [{i}/{iters}] Train AUC={tr_auc:.4f} | Val AUC={va_auc:.4f}")
        else:
            print(f"Epoch {epoch} [{i}/{iters}] Train AUC={tr_auc:.4f}")

    # —— epoch 结束：统计一次 Val AUC
    epoch_val_auc = roc_auc_score(y_val, clf.decision_function(X_val))
    epoch_train_auc = np.nanmean(epoch_train_aucs)
    print(f"\nEpoch {epoch} 结果: Train AUC={epoch_train_auc:.4f} | Val AUC={epoch_val_auc:.4f}")

    # 1) 保存“last”（当前 epoch 结束后的模型）
    dump_atomic(clf, LAST_PATH)
    print(f"📝 已保存當前最後模型（last）: {LAST_PATH}")

    # 2) 若更优，刷新并保存“best”
    if epoch_val_auc > best_val_auc:
        best_val_auc = epoch_val_auc
        best_epoch = epoch
        best_clf = copy.deepcopy(clf)
        dump_atomic(best_clf, BEST_PATH)
        print(f"🎯 新的最佳模型! Val AUC: {best_val_auc:.4f} (Epoch {epoch})")
        print(f"💾 最佳模型已保存: {BEST_PATH}")

    print(f"当前最佳: Epoch {best_epoch}, Val AUC: {best_val_auc:.4f}")

# ---------- 训练完成：保存 final、LDA，并把 best 载入内存 ----------
print(f"\n=== 训练完成 ===")
print(f"最佳模型: Epoch {best_epoch}, Val AUC: {best_val_auc:.4f}")

dump_atomic(clf, FINAL_PATH)  # 注意：final = 整个训练流程结束时的“最后模型”（不等同于最佳）
print(f"✅ 最终模型已保存: {FINAL_PATH}")

# LDA 组件（若有就保存、没有也无所谓）
try:
    dump_atomic({'lda_vec': lda_vectorizer, 'lda_model': lda_model}, LDA_PATH)
    print(f"✅ LDA组件已保存: {LDA_PATH}")
except Exception as e:
    print("保存 LDA 组件失败（可忽略）:", e)

# 加载最佳模型用于后续使用
if best_clf is not None:
    clf = best_clf
    print(f"✅ 已加载最佳模型 (Epoch {best_epoch}) 用于后续预测")
else:
    print("⚠️ 未产生更优模型，clf 保持为最后一次状态")



=== Epoch 1/50 ===
Epoch 1 [1/26] Train AUC=0.7679 | Val AUC=0.5480
Epoch 1 [2/26] Train AUC=0.7124 | Val AUC=0.5711
Epoch 1 [3/26] Train AUC=0.7172 | Val AUC=0.5728
Epoch 1 [4/26] Train AUC=0.7248 | Val AUC=0.5983
Epoch 1 [5/26] Train AUC=0.6885 | Val AUC=0.6036
Epoch 1 [6/26] Train AUC=0.7010 | Val AUC=0.5962
Epoch 1 [7/26] Train AUC=0.7043 | Val AUC=0.6081
Epoch 1 [8/26] Train AUC=0.6831 | Val AUC=0.5980
Epoch 1 [9/26] Train AUC=0.6754 | Val AUC=0.5908
Epoch 1 [10/26] Train AUC=0.7085 | Val AUC=0.5919
Epoch 1 [11/26] Train AUC=0.6503 | Val AUC=0.6099
Epoch 1 [12/26] Train AUC=0.6637 | Val AUC=0.5960
Epoch 1 [13/26] Train AUC=0.6443 | Val AUC=0.6093
Epoch 1 [14/26] Train AUC=0.6541 | Val AUC=0.6218
Epoch 1 [15/26] Train AUC=0.6824 | Val AUC=0.6229
Epoch 1 [16/26] Train AUC=0.6701 | Val AUC=0.6137
Epoch 1 [17/26] Train AUC=0.6868 | Val AUC=0.6145
Epoch 1 [18/26] Train AUC=0.6831 | Val AUC=0.5969
Epoch 1 [19/26] Train AUC=0.6683 | Val AUC=0.5995
Epoch 1 [20/26] Train AUC=0.6423 | Val 

KeyboardInterrupt: 

In [89]:
import sys, os, _pickle as pkl
from pathlib import Path
import pandas as pd
import numpy as np
from scipy.special import expit

# 1) 兼容 shim：某些 pickle 会去找顶层模块 "_loss"
try:
    import sklearn._loss as skl__loss
    sys.modules['_loss'] = skl__loss     # 把 sklearn._loss 映射成顶层 _loss，供 pickle 找
except Exception:
    pass

OUT_DIR = './output'
TEST_PATH = './dataset/test.csv'

# 2) 优先加载“最後一次(last)”；找不到再回退
candidates = ['clf-sgd-best.pkl', 'clf-sgd-final.pkl', 'clf-sgd-last.pkl', 'clf-sgd.pkl']
model_path = None
for name in candidates:
    p = Path(OUT_DIR) / name
    if p.exists():
        model_path = str(p)
        break

if model_path is None:
    raise FileNotFoundError("没有找到模型文件：./output/{clf-sgd-last.pkl, clf-sgd-final.pkl, clf-sgd-best.pkl, clf-sgd.pkl}")

with open(model_path, 'rb') as f:
    clf = pkl.load(f)
print("Loaded model (prefer last):", model_path)

# 3) （可选）LDA pack：没有就跳过
lda_vectorizer = None
lda_model = None
try:
    with open(f'{OUT_DIR}/lda.pkl','rb') as f:
        lda_pack = pkl.load(f)
        lda_vectorizer = lda_pack.get('lda_vec', None)
        lda_model = lda_pack.get('lda_model', None)
    print("Loaded LDA pack:", f'{OUT_DIR}/lda.pkl')
except Exception as e:
    print("LDA pack not loaded (ok to ignore):", e)

# 4) 推理
df_test = pd.read_csv(TEST_PATH)

B = 2000
preds = []
for i in range(0, len(df_test), B):
    sl = df_test['Page content'].iloc[i:i+B].astype(str)
    Xt = featurize_split(sl, lda_vectorizer, lda_model, n_jobs=1)  # 你训练时的同名函数
    if hasattr(clf, "decision_function"):
        probs = expit(clf.decision_function(Xt))
    elif hasattr(clf, "predict_proba"):
        probs = clf.predict_proba(Xt)[:, 1]
    else:
        raise RuntimeError("Loaded classifier has neither decision_function nor predict_proba.")
    preds.append(probs)

test_pred = np.concatenate(preds)
submission = pd.DataFrame({'Id': df_test['Id'], 'Popularity': test_pred})
sub_path = f'{OUT_DIR}/submission_90.csv'
submission.to_csv(sub_path, index=False)
print("Saved:", sub_path)


Loaded model (prefer last): output\clf-sgd-best.pkl
Loaded LDA pack: ./output/lda.pkl
Saved: ./output/submission_90.csv
